In [1]:
!pip install -q joeynmt==2.3.0


In [2]:
!pip show torch

Name: torch
Version: 2.1.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, catalyst, easyocr, fastai, joeynmt, kornia, pytorch-ignite, pytorch-lightning, stable-baselines3, timm, torchaudio, torchdata, torchmetrics, torchtext, torchvision


In [3]:
import os,re
os.environ["WANDB_DISABLED"] = "true"
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
src_lang = 'dyu'
trg_lang = "fr"
chars_to_remove_regex = '[!"&\(\),-./:;=?+.\n\[\]]'
def remove_special_characters(text):
    text = re.sub(chars_to_remove_regex, ' ', text.lower())
    return text.strip()

def clean_text(batch):
    # process source text
    batch['translation'][src_lang] = remove_special_characters(batch['translation'][src_lang])
    # process target text
    batch['translation'][trg_lang] = remove_special_characters(batch['translation'][trg_lang])

    return batch

In [6]:
from datasets import load_dataset

data = load_dataset("uvci/Koumankan_mt_dyu_fr", token=HF_TOKEN)
data = data.map(clean_text)

Generating train split:   0%|          | 0/8065 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1471 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1393 [00:00<?, ? examples/s]

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

In [7]:
display(data['train'])
display(data['validation'])
display(data['test'])

Dataset({
    features: ['ID', 'translation'],
    num_rows: 8065
})

Dataset({
    features: ['ID', 'translation'],
    num_rows: 1471
})

Dataset({
    features: ['ID', 'translation'],
    num_rows: 1393
})

In [8]:
data_dir = "./data/dyu_fr"
data.save_to_disk(data_dir)

Saving the dataset (0/1 shards):   0%|          | 0/8065 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1471 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1393 [00:00<?, ? examples/s]

In [9]:
from pathlib import Path

# model dir
model_dir = "./saved_model/dyu_fr"

# Create the config
config = """
name: "dyu_fr_transformer-sp"
joeynmt_version: "2.3.0"
model_dir: "{model_dir}"
use_cuda: True # False for CPU training
fp16: False

data:
    train: "{data_dir}"
    dev: "{data_dir}"
    test: "{data_dir}"
    dataset_type: "huggingface"
    dataset_cfg:
        name: "dyu-fr"
    sample_dev_subset: 1460
    src:
        lang: "dyu"
        max_length: 100
        lowercase: False
        normalize: False
        level: "bpe"
        voc_limit: 4000
        voc_min_freq: 1
        voc_file: "{data_dir}/vocab.txt"
        tokenizer_type: "sentencepiece"
        tokenizer_cfg:
            model_file: "{data_dir}/sp.model"
    trg:
        lang: "fr"
        max_length: 100
        lowercase: False
        normalize: False
        level: "bpe"
        voc_limit: 4000
        voc_min_freq: 1
        voc_file: "{data_dir}/vocab.txt"
        tokenizer_type: "sentencepiece"
        tokenizer_cfg:
            model_file: "{data_dir}/sp.model"
    special_symbols:
        unk_token: "<unk>"
        unk_id: 0
        pad_token: "<pad>"
        pad_id: 1
        bos_token: "<s>"
        bos_id: 2
        eos_token: "</s>"
        eos_id: 3

""".format(data_dir=data_dir, model_dir=model_dir)
with (Path(data_dir) / "config.yaml").open('w') as f:
    f.write(config)


In [10]:
!wget https://raw.githubusercontent.com/joeynmt/joeynmt/v2.3/scripts/build_vocab.py
! sudo chmod 777 build_vocab.py

--2024-08-27 14:37:38--  https://raw.githubusercontent.com/joeynmt/joeynmt/v2.3/scripts/build_vocab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13170 (13K) [text/plain]
Saving to: 'build_vocab.py'

build_vocab.py      100%[===================>]  12.86K  --.-KB/s    in 0s      

2024-08-27 14:37:38 (62.1 MB/s) - 'build_vocab.py' saved [13170/13170]



In [11]:
!python build_vocab.py {data_dir}/config.yaml --joint

2024-08-27 14:37:46.860276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 14:37:46.860406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 14:37:46.993799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Filter: 100%|█████████████████████| 8065/8065 [00:00<00:00, 20384.11 examples/s]
### Training sentencepiece...
sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/tmp/sentencepiece_5x44_tcl.txt --model_prefix=data/dyu_fr/sp --model_type=unigram --vocab_size=4000 --character_coverage=1.0 --accept_language=dyu,fr --unk_piece=<unk> --bos_piece=<s> --eo

In [12]:
config += """
testing:
    #load_model: "{model_dir}/best.ckpt"
    n_best: 1
    beam_size: 5
    beam_alpha: 1.0
    batch_size: 256
    batch_type: "token"
    max_output_length: 100
    eval_metrics: ["bleu"]
    #return_prob: "hyp"
    #return_attention: False
    sacrebleu_cfg:
        tokenize: "13a"

training:
    #load_model: "{model_dir}/latest.ckpt"
    #reset_best_ckpt: False
    #reset_scheduler: False
    #reset_optimizer: False
    #reset_iter_state: False
    random_seed: 42
    optimizer: "adamw"
    normalization: "tokens"
    adam_betas: [0.9, 0.999]
    scheduling: "warmupinversesquareroot"
    learning_rate_warmup: 500
    learning_rate: 0.0008
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.01
    loss: "crossentropy"
    batch_size: 512
    batch_type: "token"
    batch_multiplier: 4
    early_stopping_metric: "bleu"
    epochs: 300
    validation_freq: 500
    logging_freq: 5
    overwrite: True
    shuffle: True
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3

model:
    initializer: "xavier_uniform"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier_uniform"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 8
        num_heads: 4
        embeddings:
            embedding_dim: 256
            scale: True
            dropout: 0.1
        # typically ff_size = 4 x hidden_size
        hidden_size: 256
        ff_size: 1024
        dropout: 0.3
        layer_norm: "pre"
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 256
            scale: True
            dropout: 0.0
        # typically ff_size = 4 x hidden_size
        hidden_size: 256
        ff_size: 1024
        dropout: 0.1
        layer_norm: "pre"

""".format(model_dir=model_dir)
with (Path(data_dir) / "config.yaml").open('w') as f:
    f.write(config)


In [13]:
!python -m joeynmt train {data_dir}/config.yaml --skip-test

2024-08-27 14:38:06.279194: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 14:38:06.279258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 14:38:06.280680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 14:38:10,135 - INFO - joeynmt.config -                           cfg.name : dyu_fr_transformer-sp
2024-08-27 14:38:10,135 - INFO - joeynmt.config -                cfg.joeynmt_version : 2.3.0
2024-08-27 14:38:10,135 - INFO - joeynmt.config -                      cfg.model_dir : ./saved_model/dyu_fr
2024-08-27 14:38:10,135 - INFO - joeynmt.config -      

In [14]:
# Add the best model info on config file
with (Path(model_dir) / "config.yaml").open('r') as f:
    config = f.read()
resume_config = config\
  .replace(f'#load_model: "{model_dir}/best.ckpt"',
           f'load_model: "{model_dir}/best.ckpt"')

resume_config = resume_config\
  .replace(f'model_file: "{data_dir}/sp.model"',
           f'model_file: "{model_dir}/sp.model"')

resume_config = resume_config\
  .replace(f'voc_file: "{data_dir}/vocab.txt"',
           f'voc_file: "{model_dir}/vocab.txt"')

with (Path(model_dir) / "config.yaml").open('w') as f:
    f.write(resume_config)

In [15]:
!cp {data_dir}/vocab.txt  {model_dir}